<a href="https://colab.research.google.com/github/rogerallen741/PyTorch-Learning/blob/master/Lecture_10_MINIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
from torch.autograd import Variable
import time

In [112]:
#Training settings
batch_size = 64
lr_rate = 0.01
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST　Model on {device}\n{"=" * 44}')

Training MNIST　Model on cpu


In [0]:
#MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc = nn.Linear(320, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = nn.functional.relu(self.mp(self.conv1(x)))  
    x = nn.functional.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1) # flatten the tensor
    
    x = self.fc(x)
    return  nn.functional.softmax(x) 

In [0]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr_rate, momentum=0.5)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = Variable(data,volatile=True), Variable(target)
    output = model(data)
    # sum up batch loss 
    test_loss += criterion(output, target).item()   
    # get the index of the max
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)  
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [118]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 5):
        epoch_start = time.time()

        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.304322


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.302756
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.303360
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.302276
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.300730
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.301801
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.300331
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.302675
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.302205
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.302462
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.301515
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.300725
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.300614
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.301218
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.302459
Train Epoch: 1 | Batch Status: 9600/60000 (16%) | Loss: 2.300503
Train Epoch: 1 | Batch Status: 102

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Test set: Average loss: 0.0002, Accuracy: 24/10000 (0%)
Test set: Average loss: 0.0002, Accuracy: 46/10000 (0%)
Test set: Average loss: 0.0002, Accuracy: 71/10000 (1%)
Test set: Average loss: 0.0002, Accuracy: 91/10000 (1%)
Test set: Average loss: 0.0002, Accuracy: 113/10000 (1%)
Test set: Average loss: 0.0002, Accuracy: 138/10000 (1%)
Test set: Average loss: 0.0002, Accuracy: 157/10000 (2%)
Test set: Average loss: 0.0002, Accuracy: 175/10000 (2%)
Test set: Average loss: 0.0002, Accuracy: 192/10000 (2%)
Test set: Average loss: 0.0002, Accuracy: 211/10000 (2%)
Test set: Average loss: 0.0002, Accuracy: 237/10000 (2%)
Test set: Average loss: 0.0002, Accuracy: 259/10000 (3%)
Test set: Average loss: 0.0002, Accuracy: 279/10000 (3%)
Test set: Average loss: 0.0002, Accuracy: 299/10000 (3%)
Test set: Average loss: 0.0002, Accuracy: 316/10000 (3%)
Test set: Average loss: 0.0002, Accuracy: 338/10000 (3%)
Test set: Average loss: 0.0002, Accuracy: 354/10000 (4%)
Test set: Average loss: 0.0002, Acc